# PART1 
## 1. Start by creating a new Notebook for this assignment.

In [57]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#scrape table by BeautifulSoup
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv
print('libary & BeautifulSoup & csv installed')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

libary & BeautifulSoup & csv installed


# 2. Scrape table from Wikipedia page and create dataframe


In [60]:
#Secure Sockets Layer (SSL) errors ignored
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
print('SSL certificate errors ignored.')


SSL certificate errors ignored.


In [62]:
# processing HTML in the Python
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
web= requests.get(url)
source= web.text
# "lxml" is the most feature-rich and easy-to-use library for processing XML and HTML in the Python
soup= BeautifulSoup(source, 'lxml')
print('soup ready')


soup ready


In [66]:
#find a table in a Wikipedia page
table = soup.find('table',{'class':'wikitable sortable'})


In [67]:
#find rows in a Wikipedia table, information get from inspect of the page
table_rows = table.find_all('tr')


In [69]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')]) 
    #Leading and trailing whitespaces are removed by .strip()

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

print(df.head(12))


   PostalCode           Borough                                Neighbourhood
1         M1A      Not assigned                                 Not assigned
2         M2A      Not assigned                                 Not assigned
3         M3A        North York                                    Parkwoods
4         M4A        North York                             Victoria Village
5         M5A  Downtown Toronto                    Regent Park, Harbourfront
6         M6A        North York             Lawrence Manor, Lawrence Heights
7         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
8         M8A      Not assigned                                 Not assigned
9         M9A         Etobicoke      Islington Avenue, Humber Valley Village
10        M1B       Scarborough                               Malvern, Rouge
11        M2B      Not assigned                                 Not assigned
12        M3B        North York                                    Don Mills

# 3. Clean dataframe

~The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.

~Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

~More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

~If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

~Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

~In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [70]:
#Ignore cells with a borough that is Not assigned
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()


,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [81]:
df1 = df.reset_index(drop=True)
df1.head(12)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [79]:
df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostalCode     103 non-null    object
 1   Borough        103 non-null    object
 2   Neighbourhood  103 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


#~More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


In [83]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x)) 
# group by postalcode, aggragate colums by joint, use "," to seperate each value after aggragating
df2 = df1.reset_index(drop=True)
df2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


~If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [85]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

df2.head(12)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


~In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [86]:
df2.shape

(103, 3)

# Part2

## Get the latitude and the longitude coordinates of each neighborhood. 

Since the given package is not reliable, i will use the csv file to create the requested dataframe


In [117]:
# get Geospatial_Coordinates.csv
Geotb=pd.read_csv('/Users/shanshanbai/Downloads/Geospatial_Coordinates.csv')
Geotb.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [118]:
Geotb.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
Geotb.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [120]:
df_geotb = pd.DataFrame(data=Geotb)

In [121]:
df_geotb.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [123]:
merged_inner = pd.merge(left=df2, right=df_geotb, left_on='PostalCode', right_on='PostalCode')
merged_inner.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [124]:
merged_inner.shape

(103, 5)